In [1]:
from QuICT.core import Circuit
from QuICT.core.utils import GateType
from QuICT.simulation import Simulator

In [ ]:
# Build circuit with 100 random gates and 5 qubits
circuit = Circuit(5)
type_list = [GateType.x, GateType.h, GateType.rx, GateType.ry, GateType.rz, GateType.cx]
circuit.random_append(rand_size=100, typelist=type_list)

# Initial Simulator
simulator = Simulator(
    device="GPU",
    backend="statevector",
    shots=10,
    precision="double"
)
result = simulator.run(circuit)    # get simulation's result

In [ ]:
# Initial unitary simulator
unitary_simulator = Simulator(
    device="CPU",
    backend="unitary",
    shots=1
)
result = simulator.run(circuit)    # get simulation's result

In [ ]:
# Using multi-GPU simulator
from concurrent.futures import ProcessPoolExecutor, as_completed
from cupy.cuda import nccl

from QuICT.utility import Proxy
from QuICT.simulation.gpu_simulator import MultiStateVectorSimulator

def worker_thread(ndev, uid, dev_id):
    proxy = Proxy(ndevs=ndev, uid=uid, dev_id=dev_id)

    simulator = MultiStateVectorSimulator(
        proxy=proxy,
        precision="double",
        gpu_device_id=dev_id,
        sync=True
    )
    state = simulator.run(cir)
    return state


ndev = 2    # Device number
uid = nccl.get_unique_id()    # generate nccl id for Proxy connection
with ProcessPoolExecutor(max_workers=ndev) as executor:
    tasks = [
        executor.submit(worker_thread, ndev, uid, dev_id) for dev_id in range(ndev)
    ]

# Collect result from each device
results = []
for t in as_completed(tasks):
    results.append(t.result())


In [ ]:
# Initial remote simulator
simulator = Simulator(
    device="qcompute",
    backend="cloud_baidu_sim2_earth",
    shots=10,
    token=token_baidu
)
result = simulator.run(circuit)    # get simulation's result